# Storing trained models
This notebook contains the same sample as we used in Chapter 6, Working with timeseries data. It trains a recurrent neural network containing LSTM recurrent units on solarpanel data. This notebook is an extended version of the original one with checkpointing. We've left the original comments in for clarity.

In [4]:
from cntk.losses import squared_error
from cntk.io import CTFDeserializer, MinibatchSource, INFINITELY_REPEAT, StreamDefs, StreamDef
from cntk.learners import adam
from cntk.logging import ProgressPrinter
from cntk.train import TestConfig, CheckpointConfig

The notebook uses a set of constants to control various settings.
The most important settings are the batch size, epoch size and number of epochs to train for.

We've normalized the training data based on the maximum total power generated by the solar panel. 
This value is stored as a constant here to denormalize the output of the neural network normal usage.

In [5]:
BATCH_SIZE = 14 * 10
EPOCH_SIZE = 12434
EPOCHS = 10

# This value is required to convert the normalized values back to their original value.
# You can obtain this value by looking at the maximum value for the solar.total column
NORMALIZE = 19100

## Building the model
The model we're using is a recurrent neural network with an LSTM as the implementation for the recurrent layer in the network. We've wrapped the LSTM in a Fold layer because we're only interested in the final output of the recurrent layer. 
The output of the network is generated using a final Dense layer.

Note, the input features for the model are stored in a sequence input variable. This is required since we're working with sequences rather than single samples. The target output is stored in a regular input variable as we're only interested in predicting a single output.

In [6]:
from cntk import sequence, default_options, input_variable
from cntk.layers import Recurrence, LSTM, Dropout, Dense, Sequential, Fold

features = sequence.input_variable(1)

with default_options(initial_state = 0.1):
    model = Sequential([
        Fold(LSTM(15)),
        Dense(1)
    ])(features)
    
target = input_variable(1, dynamic_axes=model.dynamic_axes)

## Training the model
The model is trained using a mean squared error loss function. The data for the model is coming from a set of CTF Files containing sequences of measurements per day. 

In [7]:
from cntk import Function

@Function
def criterion_factory(z, t):
    loss = squared_error(z, t)
    metric = squared_error(z, t)    
    
    return loss, metric

loss = criterion_factory(model, target)
learner = adam(model.parameters, lr=0.005, momentum=0.9)

In order to load data into the training process we need to deserialize sequences from a set of CTF files. The `create_datasource` function is a useful utility function to create both the training and test datasources. 

In [8]:
def create_datasource(filename, sweeps=INFINITELY_REPEAT):
    target_stream = StreamDef(field='target', shape=1, is_sparse=False)
    features_stream = StreamDef(field='features', shape=1, is_sparse=False)

    deserializer = CTFDeserializer(filename, StreamDefs(features=features_stream, target=target_stream))
    datasource = MinibatchSource(deserializer, randomize=True, max_sweeps=sweeps)    
    
    return datasource

In [9]:
train_datasource = create_datasource('solar_train.ctf')
test_datasource = create_datasource('solar_val.ctf', sweeps=1)

Now that we've setup the data sources, model, and loss function let's start the training process.
Please be aware, this takes a long time on a computer with just a CPU. If you can, use a GPU to train this model.

**Note** Checkpointing is enabled in this version of the notebook, so you can interrupt training at any point. When you restart the notebook at a later time you can execute all cells again and it will continue training where it left of the last time.

In [10]:
progress_writer = ProgressPrinter(0)
test_config = TestConfig(test_datasource)

input_map = {
    features: train_datasource.streams.features,
    target: train_datasource.streams.target
}

checkpoint_config = CheckpointConfig('solar.dnn', frequency=100, restore=True, preserve_all=False)

history = loss.train(
    train_datasource, 
    epoch_size=EPOCH_SIZE,
    parameter_learners=[learner], 
    model_inputs_to_streams=input_map,
    callbacks=[progress_writer, test_config, checkpoint_config],
    minibatch_size=BATCH_SIZE,
    max_epochs=EPOCHS)

 average      since    average      since      examples
    loss       last     metric       last              
 ------------------------------------------------------
Finished Evaluation [1]: Minibatch[1-598]: metric = 0.86% * 2239;


## Storing the model in ONNX format
When you're done training you can store the model on disk in the ONNX format so you can use it from, for example, C# or Java.

In [11]:
from cntk import ModelFormat

model.save('solar.onnx', format=ModelFormat.ONNX)